In [1]:
import requests
import math
import os
import csv

## Functions

In [2]:
def getRow(trip):
    '''
    Flattens a BlaBlaCar trip object into the plain list we want to upload to CARTO
    '''
    return {
            'url'     : trip['links']['_front'], 
            'from_lat': trip['departure_place']['latitude'], 
            'from_lon': trip['departure_place']['longitude'], 
            'to_lat'  : trip['arrival_place']['latitude'],
            'to_lon'  : trip['arrival_place']['longitude'],
            'car_comfort' : trip['car']['comfort_nb_star'] if 'car' in trip else '' ,
            'car_maker'   : trip['car']['make'] if 'car' in trip else '',
            'dep_date'    : trip['departure_date'],
            'distance'    : trip['distance']['value'],
            'duration'    : trip['duration']['value'],
            'price'       : trip['price_with_commission']['value'],
            'seats'       : trip['seats']
            }

In [3]:
def getTrips(city):
    '''
    Iterates over the BlaBlaCar /trips/ endpoint to retrieve all the pages
    '''
    trips = []
    page = 1
    
    URL = "https://public-api.blablacar.com/api/v3/trips?key=YOUR_API_KEY&from_coordinate=48.864716%2C2.349014&from_country=FR&to_coordinate=51.509865%2C-0.118092&to_country=GB&locale=fr-FR&currency=EUR&from_cursor=cGFnZT0xOA%3D%3D&start_date_local=2019-01-10T08%3A55%3A00"
    KEY = os.getenv('BLABLACAR_API')
    CUR = "EUR"
    HEADERS = {
        'Content-Type': "application/json",
        'Cache-Control': "no-cache"
    }
    QS_BASE = {"key": KEY,"cur": CUR, "limit": 100} 

    querystring = dict( { 'page' : page, "fn": city, }, **QS_BASE)
    response = requests.request("GET", URL, headers=HEADERS, params=querystring)
    rj = response.json()

    while 'trips' in rj:
        #print('  Got page {}'.format(page))
        trips.extend(rj['trips'])
        page = page + 1

        querystring = dict( { 'page' : page, "fn": city }, **QS_BASE)
        response = requests.request("GET", URL, headers=HEADERS, params=querystring)
        rj = response.json()
        
    return trips


## Process cities

Process a list of cities to get all the trips and generates as a result a CSV ready to be used at CARTO

In [4]:
cities = ['Valencia','Madrid','Barcelona','Bilbao','Sevilla']

In [5]:
all_trips = []

for city in cities:
    print('Getting the trips from BlaBlaCar API for {}...'.format(city))
    trips = getTrips(city)        
    print('{} trips retreived'.format(len(trips)))        
    all_trips.append({ 'city':city, 'trips': trips})
    print('')

Getting the trips from BlaBlaCar API for Valencia...
0 trips retreived

Getting the trips from BlaBlaCar API for Madrid...
0 trips retreived

Getting the trips from BlaBlaCar API for Barcelona...
0 trips retreived

Getting the trips from BlaBlaCar API for Bilbao...
0 trips retreived

Getting the trips from BlaBlaCar API for Sevilla...
0 trips retreived



Once get all the trips, save them into a single CSV file

In [6]:
filename = 'blablatrips.csv'
print('Generating {}...'.format(filename))
with open(filename, 'w', newline='') as csvfile:
    fieldnames = [
        'city', 'url', 'from_lat', 'from_lon', 'to_lat', 'to_lon', 'car_comfort', 
        'car_maker', 'dep_date', 'distance', 'duration', 'price', 'seats'
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for group in all_trips:
        city = group['city']
        trips = group['trips']
        try:
            for trip in trips:
                row = getRow(trip)
                writer.writerow(dict({'city':city},**row))
        except KeyError as e:
            print(e)
            print(trip)

print('Done!')

Generating blablatrips.csv...
Done!
